In [5]:
using SwiftObjectStores
using SwiftObjectStores.stat
using Base.Test
using JLD
import PyCall.PyError
ENV["OS_PASSWORD"] = "MzhkYzg0M2IwMTU2NGU1"
ENV["OS_USERNAME"] = "20361362@student.uwa.edu.au"

const serv = SwiftService(
    auth_url =  "https://keystone.rc.nectar.org.au:5000/v2.0/",
    tenant_name = "UWA_SEMANTIC_VECTORS", 
    #It is a little scary that I am testing in the same Tanant as I am using for my actual research
)

PyObject <swiftclient.service.SwiftService object at 0x7f2816cb3dd0>

In [6]:
list(serv)

Dict{String,Dict{Any,Any}} with 5 entries:
  "sensemodels" => Dict{Any,Any}(Pair{Any,Any}("name","sensemodels"),Pair{Any,Any}("bytes",2362834369),Pair{Any,Any}("count",20))
  "testing-out-swift-container_segments" => Dict{Any,Any}(Pair{Any,Any}("name","testing-out-swift-container_segments"),Pair{Any,Any}("bytes",0),Pair{Any,Any}("count",0))
  "sensemodels_segments" => Dict{Any,Any}(Pair{Any,Any}("name","sensemodels_segments"),Pair{Any,Any}("bytes",20710368140),Pair{Any,Any}("count",21))
  "swift.jl-testing_segments" => Dict{Any,Any}(Pair{Any,Any}("name","swift.jl-testing_segments"),Pair{Any,Any}("bytes",0),Pair{Any,Any}("count",0))
  "testing-out-swift-container" => Dict{Any,Any}(Pair{Any,Any}("name","testing-out-swift-container"),Pair{Any,Any}("bytes",58746),Pair{Any,Any}("count",14))

In [7]:
;cat ~/openrc.sh

#!/bin/bash
# To use an Openstack cloud you need to authenticate against keystone, 
# which returns a **Token** and **Service Catalog**.  The catalog 
# contains the endpoint for all services the user/tenant has access to - 
# including nova, glance, keystone, swift.
#
# *NOTE*: Using the 2.0 *auth api* does not mean that compute api is 
# 2.0.  We will use the 1.1 *compute api*
export OS_AUTH_URL=https://keystone.rc.nectar.org.au:5000/v2.0/
# With the addition of Keystone we have standardized on the term 
# **tenant** as the entity that owns the resources.
export OS_TENANT_ID=eca532d8c1e84913bd21d6d22b30f985
export OS_TENANT_NAME="UWA_SEMANTIC_VECTORS"
# In addition to the owning entity (tenant), openstack stores the entity 
# performing the action as the **user**.
export OS_USERNAME="20361362@student.uwa.edu.au"
# With Keystone you pass the keystone password.
export OS_PASSWORD="MzhkYzg0M2IwMTU2NGU1"
# If your configuration has multiple regions, we set that information 
# here. OS_RE

"20361362@student.uwa.edu.au"

In [4]:
using SwiftObjectStores
using SwiftObjectStores.stat
using Base.Test
using JLD
import PyCall.PyError

const serv = SwiftService(
    auth_url =  "https://keystone.rc.nectar.org.au:5000/v2.0/",
    tenant_name = "UWA_SEMANTIC_VECTORS", 
    #It is a little scary that I am testing in the same Tanant as I am using for my actual research
)
const container_name = "swift.jl-testing"

if haskey(SwiftObjectStores.list(serv), container_name)
    warn("Past test left mess. Deleting " * container_name * " for use in testing")
    delete(serv,container_name)    
end


@testset "PreCreate" begin
    @test_throws PyError stat(serv, container_name)  # container should not exist
end



@testset "Stream User Path" begin
    const obj_name = "eg1buffer"
    
    eg_text = "The Julia Language: A fresh approach to technical computing. λ=x²"
    @test put_file(serv, container_name, obj_name, IOBuffer(eg_text)) != nothing
    get_file(serv, container_name, obj_name) do fp
        @test readstring(fp) == eg_text
    end
    
    @test Dict(stat(serv, container_name)["items"])["Objects"] == "1"
    @test stat(serv, container_name, obj_name) !=nothing
    
    @test delete(serv,container_name, obj_name)!=nothing
    @test stat(serv, container_name, obj_name)["success"] == false 
    @test stat(serv, container_name, "eg1buffer")["error"][:http_status] == 404
end;


@testset "File User Path" begin
    const obj_name = "eg2file"
    const file_name = joinpath(@__FILE__, "eg2file.txt")
    open(file_name, "w") do fp
        println(fp, """
        Call me Ishmael. Some years ago- never mind how long precisely- 
        having little or no money in my purse, and nothing particular to interest me on shore,
        I thought I would sail about a little and see the watery part of the world. 
        It is a way I have of driving off the spleen and regulating the circulation. 
        """)
    end
    
    @test put_file(serv, container_name, obj_name, file_name) != nothing
    @test validate_file(serv, container_name, obj_name, file_name) 
end


@testset "File User Path with PseudoDir" begin
    const obj_name = "eg2file"
    const file_name = joinpath(@__FILE__, "eg3file.txt")
    open(file_name, "w") do fp
        println(fp, """
        There now is your insular city of the Manhattoes,
        belted round by wharves as Indian isles by coral reefs 
        - commerce surrounds it with her surf. 
        Right and left, the streets take you waterward. 
        Its extreme downtown is the battery, 
        where that noble mole is washed by waves,
        and cooled by breezes, 
        which a few hours previous were out of sight of land.
        Look at the crowds of water-gazers there
        """)
    end
    
    @test put_file(serv, container_name*"/psubdir", obj_name, file_name) != nothing
    @test validate_file(serv, container_name*"/psubdir", obj_name, file_name) 
end


immutable Mt
    m::String
    v::Int64
end

@testset "JLD" begin
    const obj_name = "eg2jld"
    val = Mt("Hello Dearest", 52)
    
    @test put_jld(serv, container_name, obj_name, mm=val) != nothing
    @test get_jld(serv, container_name, obj_name, "mm").m == val.m
    @test get_jld(serv, container_name, obj_name, "mm").v == val.v
end


@testset "Post_test" begin
    @test delete(serv,container_name)[end]["success"] == true
    @test_throws PyError stat(serv, container_name) 
    @test haskey(SwiftObjectStores.list(serv), container_name) == false
end;

Test Summary: | Pass  Total
  PreCreate   |    1      1
Test Summary:    | Pass  Total
  Stream User Path |    7      7
Test Summary:  | Pass  Total
  File User Path |    2      2
Test Summary:                 | Pass  Total
  File User Path with PseudoDir |    2      2
Test Summary: | Pass  Total
  JLD         |    3      3
Test Summary: | Pass  Total
  Post_test   |    3      3


Test Summary:  | Pass  Total
  File User Path |    2      2


Base.Test.DefaultTestSet("File User Path",Any[Test Passed
  Expression: put_file(serv,container_name,obj_name,file_name) != nothing
   Evaluated: Dict{Any,Any}(Pair{Any,Any}("response_dict",Dict{Any,Any}(Pair{Any,Any}("headers",Dict{Any,Any}(Pair{Any,Any}("x-trans-id","tx499ae7d8e3ac402c883ac-0057d19d6a"),Pair{Any,Any}("etag","048edbaf9c952a7c511784905d3ab77a"),Pair{Any,Any}("content-type","text/html; charset=UTF-8"),Pair{Any,Any}("date","Thu, 08 Sep 2016 17:18:34 GMT"),Pair{Any,Any}("last-modified","Thu, 08 Sep 2016 17:18:35 GMT"),Pair{Any,Any}("content-length","0"))),Pair{Any,Any}("status",201),Pair{Any,Any}("response_dicts",Any[Dict{Any,Any}(Pair{Any,Any}("headers",Dict{Any,Any}(Pair{Any,Any}("x-trans-id","tx499ae7d8e3ac402c883ac-0057d19d6a"),Pair{Any,Any}("etag","048edbaf9c952a7c511784905d3ab77a"),Pair{Any,Any}("content-type","text/html; charset=UTF-8"),Pair{Any,Any}("date","Thu, 08 Sep 2016 17:18:34 GMT"),Pair{Any,Any}("last-modified","Thu, 08 Sep 2016 17:18:35 GMT"),Pair{Any,Any}

Test Summary:                 | Pass  Total
  File User Path with PseudoDir |    2      2


Base.Test.DefaultTestSet("File User Path with PseudoDir",Any[Test Passed
  Expression: put_file(serv,container_name * "/psubdir",obj_name,file_name) != nothing
   Evaluated: Dict{Any,Any}(Pair{Any,Any}("response_dict",Dict{Any,Any}(Pair{Any,Any}("headers",Dict{Any,Any}(Pair{Any,Any}("x-trans-id","txf62857ccf3fa48798fae2-0057d19e0e"),Pair{Any,Any}("etag","8b66e97639fd1e6a78289f1b5f1f4a3b"),Pair{Any,Any}("content-type","text/html; charset=UTF-8"),Pair{Any,Any}("date","Thu, 08 Sep 2016 17:21:18 GMT"),Pair{Any,Any}("last-modified","Thu, 08 Sep 2016 17:21:19 GMT"),Pair{Any,Any}("content-length","0"))),Pair{Any,Any}("status",201),Pair{Any,Any}("response_dicts",Any[Dict{Any,Any}(Pair{Any,Any}("headers",Dict{Any,Any}(Pair{Any,Any}("x-trans-id","txf62857ccf3fa48798fae2-0057d19e0e"),Pair{Any,Any}("etag","8b66e97639fd1e6a78289f1b5f1f4a3b"),Pair{Any,Any}("content-type","text/html; charset=UTF-8"),Pair{Any,Any}("date","Thu, 08 Sep 2016 17:21:18 GMT"),Pair{Any,Any}("last-modified","Thu, 08 Sep 2016 

Test Summary: | Pass  Total
  JLD         |    3      3


Base.Test.DefaultTestSet("JLD",Any[Test Passed
  Expression: put_jld(serv,container_name,obj_name,mm=val) != nothing
   Evaluated: Dict{Any,Any}(Pair{Any,Any}("response_dict",Dict{Any,Any}(Pair{Any,Any}("headers",Dict{Any,Any}(Pair{Any,Any}("x-trans-id","tx8615399a405448c19ed87-0057d19d8e"),Pair{Any,Any}("etag","c25527ee056386ab2c154dfc084e45c7"),Pair{Any,Any}("content-type","text/html; charset=UTF-8"),Pair{Any,Any}("date","Thu, 08 Sep 2016 17:19:10 GMT"),Pair{Any,Any}("last-modified","Thu, 08 Sep 2016 17:19:11 GMT"),Pair{Any,Any}("content-length","0"))),Pair{Any,Any}("status",201),Pair{Any,Any}("response_dicts",Any[Dict{Any,Any}(Pair{Any,Any}("headers",Dict{Any,Any}(Pair{Any,Any}("x-trans-id","tx8615399a405448c19ed87-0057d19d8e"),Pair{Any,Any}("etag","c25527ee056386ab2c154dfc084e45c7"),Pair{Any,Any}("content-type","text/html; charset=UTF-8"),Pair{Any,Any}("date","Thu, 08 Sep 2016 17:19:10 GMT"),Pair{Any,Any}("last-modified","Thu, 08 Sep 2016 17:19:11 GMT"),Pair{Any,Any}("content-lengt

Test Summary: | Pass  Total
  Post_test   |    3      3


Base.Test.DefaultTestSet("Post_test",Any[Test Passed
  Expression: ((delete(serv,container_name))[end])["success"] == true
   Evaluated: true == true,Test Passed
  Expression: stat(serv,container_name)
      Thrown: PyCall.PyError,Test Passed
  Expression: haskey(SwiftObjectStores.list(serv),container_name) == false
   Evaluated: false == false],false)

In [ ]:
y=open("eg2file.txt") do fp
    readstring(fp)
end

In [ ]:
x==y

In [ ]:
"Note this reads out the streams to their ends"
function validate_equal3!(a_stream::IO, b_stream::IO; buffer_len=4*1024)
    a_buf = Vector{UInt8}(buffer_len)
    b_buf = Vector{UInt8}(buffer_len)

    while(true)
        a_len_read = readbytes!(a_stream, a_buf)
        b_len_read = readbytes!(b_stream, b_buf)
        if a_buf[1:a_len_read] != b_buf[1:b_len_read]
            println("**")
            println([a_buf b_buf])
            
            @show a_buf[a_buf.!=b_buf],  b_buf[a_buf.!=b_buf], 
            return false
        end
        @assert(a_len_read == b_len_read)
        if a_len_read<length(a_buf)
            return true #we have reached the end
        end
    end

end

"""
Downloads a file file from Swift and checks if it the same as the local file giv
en by `fname`.
Note: this is a comprehense check. Normally you would prefer to just check Hash'
s.
This method is for testing checking that the hash checking method works.
"""
function validate_file2(serv, container::String, objectname::String, fname::String)
    get_file(serv, container, objectname; verbose=true) do rfp
        open(fname, "r") do lfp
            validate_equal2!(rfp, lfp)
        end
    end
end



In [ ]:
get_file(serv, container_name, "eg2file") do fpa
    open("eg2file.txt") do fpb
        validate_equal3!(fpa,fpb)
        #a=readbytes(fpa) 
        #b=readbytes(fpb)
        #[a b map(Char, a) map(Char, b) a.==b]
        
    end
end

In [ ]:
using PyCall
using JLD
using FileIO

In [ ]:
@pyimport swiftclient.service as service
@pyimport swiftclient.client as client
serv = service.SwiftService(Dict(
    "os_auth_url"     => "https://keystone.rc.nectar.org.au:5000/v2.0/",
    "os_username"     => "20361362@student.uwa.edu.au",
    "os_password"     => "MzhkYzg0M2IwMTU2NGU1",
    "os_tenant_name"  => "UWA_SEMANTIC_VECTORS",
    "auth_version"    => "2",
    "segment_size"    => 1024^3, #1GB
    "segment_threads" => 10,
    "object_threads"  => 24
))

In [ ]:
#A constructor for the python object
"""
Creates a swift service object, which describes the properties of your connection.
Default values for are read from enviroment variables, using the names from your `openstackrc.sh`.
Just like for the swift commandline client
"""
function SwiftService(;auth_url = ENV["OS_AUTH_URL"],
                        username::String = ENV["OS_USERNAME"],
                        password::String = ENV["OS_PASSWORD"],
                        tenant_name::String = ENV["OS_TENANT_NAME",
                        auth_version::String = "2",
                        segement_size::Integer = 1024^3,
                        segment_threads::Integer=24,
                        object_threads::Integer=24)
    service.SwiftService(Dict(
        "os_auth_url"     => auth_url,
        "os_username"     => username,
        "os_password"     => password,
        "os_tenant_name"  => tenant_name,
        "auth_version"    => auth_version,
        "segment_size"    => segment_size, #1GB
        "segment_threads" => segment_threads,
        "object_threads"  => object_threads
    ))
end

"""Defaults to reading """
function SwiftService()
    env_vars = 
    , "OS_PASSWORD", "OS_TENANT_NAME"]
    SwiftService((ENV[ev] for ev in env_vars)...)
end

In [ ]:
push!(Set(), 2,3)

In [ ]:
function show_responses(responses)
    local shown::Set
    function subshow(prefix, value) #Non-dict
        println("** $prefix **:")
        println("\t",value)
    end
    
    function subshow(prefix, dict::Dict)      
        for (key, value) in dict
            value ∈ shown && continue  # Skip any fields already shown
            push!(shown, value)
            subshow("$prefix $key", value)
        end
    end
    
    function subshow(prefix, vec::Vector)      
        for entry in vec
            entry ∈ shown && continue  # Skip any fields already shown
            push!(shown, entry)
            subshow("$prefix - ", entry)
        end
    end
        
    
    println("Swift  Responses:")
    println("-------------------------")
    for resp in responses
        shown = Set()
        push!(shown, resp)
        subshow("", resp)
        println()
    end
end

In [ ]:

"""
`container` can either be be just a container name, or a pseudofolder path
"""
function put_file(serv, container::String, name::String, fname::String; verbose::Bool=false)
    uo = service.SwiftUploadObject(fname, object_name=name)
    async_put = serv[:upload](container, [uo]) 
    responses = collect(async_put)
    if verbose
        show_responses(responses)
    end
    ret = responses[end]
    ret["success"] || error(ret["error"])
    ret
end

"""Note: this reads out the `fp` IO to the end"""
function put_file(serv, container::String, name::String, fp::IO; verbose::Bool=false)
    mktempdir() do tdir
        fname = joinpath(tdir, name)
        open(fname,"w") do fp_inner
            write(fp_inner, readbytes(fp))
        end
        put_file(serv, container, name, fname; verbose=verbose)
    end
end

"""Save data as a a JLD file, and upload to Swift"""
function put_jld(serv, container::String, name::String; data...)
    mktempdir() do tdir
        fname = joinpath(tdir, name)
        save(File(format"JLD", fname), Base.Flatten(((string(name), val) for (name,val) in data))...)
        put_file(serv, container, name, fname)
    end
end

###############

"""Download from swift, writing the result to the file given by `fname`.
    `container` can either be be just a container name, or a pseudofolder path    
"""
function get_file!(serv, container::String, name::String, fname::String; verbose::Bool=false)
    container_parts = split(strip(container, '/'), "/")
    container = container_parts[1]
    if length(container_parts) > 1
        psedudofolder = join(container_parts[2:end], "/")
        name = psedudofolder*"/"*name
    end

    async_get = serv[:download](container, [name], Dict("out_file" => fname))
    responses=collect(async_get)
    if verbose
        show_responses(responses)
    end
    ret = responses[end]
    ret["success"] || error(ret["error"])
    ret
end

"""Read IO stream, from Swift, and call func, on it, returning the result """
function get_file(func::Function, serv, container::String, name::String; verbose::Bool=false)
    mktempdir() do tdir
        fname = joinpath(tdir, name)
        get_file!(serv, container, name, fname; verbose=verbose)    
        open(func, fname,"r")
    end
end

"""Download a JLD file from Swift. `data` is a list of fieldnames to read."""
    function get_jld(serv, container::String, name::String, verbose::Bool = false, data...)
    mktempdir() do tdir
        fname = joinpath(tdir, name)
        get_file!(serv, container, name, fname; verbose=verbose)
        JLD.load(File(format"JLD", fname), data...)
    end
end

In [ ]:
get_file(serv, "sensemodels/adagrmam", "paper_v1.params.jld"; verbose=true) do fp
    println(convert(String, read(fp))[1:256])
end

In [ ]:
rr=get_jld(serv, "sensemodels/adagram", "more_senses.adagram_model.jld", true);

In [ ]:
amr=rr["am"];

In [ ]:
aml = JLD.load("../SenseSplittingWord2Vec/eval/models/adagram/more_senses.adagram_model.jld", "am");

In [ ]:
aml.vm.Out == amr.vm.Out

In [ ]:
function list(serv) 
    serv[:stat]()
end
function list(serv, containter::String)
    serv[:stat](container)
end
function list(serv, containter::String, object::String)
    serv[:stat](container, object)
end
stat = list


In [ ]:
"Note this reads out the streams to their ends"
function validate_equal!(a_stream::IO, b_stream::IO; buffer_len=4*1024)
    a_buf = Vector{UInt8}(buffer_len)
    b_buf = Vector{UInt8}(buffer_len)
    
    while(true)
        a_len_read = readbytes!(a_stream, a_buf)
        b_len_read = readbytes!(b_stream, b_buf)
        if a_buf != b_buf
            return false
        end
        @assert(a_len_read == b_len_read)
        if a_len_read<length(a_buf)
            return true #we have reached the end
        end
    end
    
end

"""
Downloads a file file from Swift and checks if it the same as the local file given by `fname`.
Note: this is a comprehense check. Normally you would prefer to just check Hash's.
This method is for testing checking that the hash checking method works.
"""
function validate_file(serv, container::String, objectname::String, fname::String)
    get_file(serv, container, objectname; verbose=true) do rfp
        open(fname, "r") do lfp
            validate_equal!(rfp, lfp) || error("$fname is not identical to $container : $objectname")
        end
    end
end

In [ ]:
validate_file(serv, "sensemodels/adagram", "more_senses.adagram_model.jld", "../SenseSplittingWord2Vec/eval/models/adagram/more_senses.adagram_model.jld")

In [ ]:
using Glob

In [ ]:
for fn in [glob(glob"../SenseSplittingWord2Vec/eval/models/adagram/*.jld");
           glob(glob"../SenseSplittingWord2Vec/eval/models/adagram/*.adagram_model")]
    @show fn
    name= basename(fn)
    put_file(serv, "sensemodels/adagram/", name, fn)
end


for fn in [glob(glob"../SenseSplittingWord2Vec/eval/models/ss/*.jld");
           glob(glob"../SenseSplittingWord2Vec/eval/models/ss/*.log")]
    @show fn
    name= basename(fn)
    put_file(serv, "sensemodels/greedy/", name, fn)
end



for fn in [glob(glob"../SenseSplittingWord2Vec/eval/models/plain/*.jld");
           glob(glob"../SenseSplittingWord2Vec/eval/models/plain/*.log")]
    @show fn
    name = basename(fn)
    put_file(serv, "sensemodels/plain/", name, fn)
end



In [ ]:
for fn in [glob(glob"../SenseSplittingWord2Vec/eval/models/adagram/*.jld");
           glob(glob"../SenseSplittingWord2Vec/eval/models/adagram/*.adagram_model")]
    @show fn
    name= basename(fn)
    validate_file(serv, "sensemodels/adagram/", name, fn)
end


for fn in [glob(glob"../SenseSplittingWord2Vec/eval/models/ss/*.jld");
           glob(glob"../SenseSplittingWord2Vec/eval/models/ss/*.log")]
    @show fn
    name= basename(fn)
    validate_file(serv, "sensemodels/greedy/", name, fn)
end



for fn in [glob(glob"../SenseSplittingWord2Vec/eval/models/plain/*.jld");
           glob(glob"../SenseSplittingWord2Vec/eval/models/plain/*.log")]
    @show fn
    name = basename(fn)
    validate_file(serv, "sensemodels/plain/", name, fn)
end



In [ ]:
; md5sum ../SenseSplittingWord2Vec/eval/models/adagram/more_senses.adagram_model.jld